# 주소 정리

## library

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import re
from tqdm import tqdm

## 데이터 로드

In [28]:
df = pd.read_csv('2020-22년_급속충전.csv', index_col=0, usecols=['주소', '지역', '시군구'])
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 11665822 entries, 강원도 to 서울특별시
Data columns (total 2 columns):
 #   Column  Dtype 
---  ------  ----- 
 0   시군구     object
 1   주소      object
dtypes: object(2)
memory usage: 267.0+ MB


In [29]:
df = df.reset_index()
df.head()

,지역,시군구,주소
0,강원도,인제군,강원도 인제군 상남면 하남리 339-7(양양방향)
1,강원도,인제군,강원도 인제군 상남면 하남리 339-7(양양방향)
2,강원도,인제군,강원도 인제군 상남면 하남리 339-7(양양방향)
3,강원도,인제군,강원도 인제군 상남면 하남리 339-7(양양방향)
4,광주광역시,북구,광주광역시 북구 첨단 벤처로 108번길 9


In [30]:
adr_uni = df['주소'].unique()
len(adr_uni)

4364

In [31]:
df2 = df.drop_duplicates(['주소']).reset_index(drop=True)
df2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4364 entries, 0 to 4363
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   지역      4364 non-null   object
 1   시군구     4364 non-null   object
 2   주소      4364 non-null   object
dtypes: object(3)
memory usage: 102.4+ KB


In [32]:
adr = df2['주소'].to_numpy()
len(adr)

4364

### 특이주소 처리

In [21]:
# 경상북도 경주시 천원2길 9 경북 경주시 교동 153-5
# 충청북도 단양군 영춘면 온달평강3로 9 온달평강3로 9-1
# 광주광역시 북구 첨단 벤처로 108번길 9

In [33]:
count = 0
for i, one in enumerate(adr):
    if '경상북도 경주시 천원2길 9 경북 경주시 교동 153-5' in one:
        adr[i] = '경상북도 경주시 교동 천원2길 9'
        count += 1
count

1

In [34]:
count = 0
for i, one in enumerate(adr):
    if '충청북도 단양군 영춘면 온달평강3로 9 온달평강3로 9-1' in one:
        adr[i] = '충청북도 단양군 영춘면 온달평강3로 9'
        count += 1
count

1

In [35]:
count = 0
for i, one in enumerate(adr):
    if '광주광역시 북구 첨단 벤처로 108번길 9' in one:
        adr[i] = '광주광역시 북구 첨단벤처로108번길 9'
        count += 1
count

1

### 빈칸 2개 -> 1개

In [36]:
count = 0
for i, one in enumerate(adr):
    if '  ' in one:
        adr[i] = one.replace('  ', ' ')
        count += 1
count

128

### 컴마(,) 제거

In [37]:
count = 0
for i, one in enumerate(adr):
    if ',' in one:
        adr[i] = one.replace(',', '')
        count += 1
count

324

### 복수 주소 오류
- 한주소에 2개 주소 들어있는 오류
- 동일지역 : 56
- 다른지역 : 2

#### 동일지역

In [38]:
np.where(adr == '전라남도 무안군 망운면 운해로 1436-10 전라남도 함평군 신광면 학동로 1398-9')

(array([2608], dtype=int64),)

In [39]:
adr[2608]

'전라남도 무안군 망운면 운해로 1436-10 전라남도 함평군 신광면 학동로 1398-9'

In [40]:
si = df2['시군구'].to_numpy()

In [41]:
count = 0
for i, one in tqdm(enumerate(adr)):
    first = list(one.split())[0]
    one = one.replace(' ' + first, '$' + first)
    multi_adr = list(one.split('$'))
    if len(multi_adr) == 2:
        count += 1
        for j, two in enumerate(multi_adr):
            if si[i] in two:
                adr[i] = multi_adr[j]
count

4364it [00:00, 436348.40it/s]


56

In [42]:
adr[2608]

'전라남도 무안군 망운면 운해로 1436-10'

#### 다른지역

In [43]:
sep = ' 광주광역시| 전라남도| 전라북도| 제주특별자치도| 세종특별자치시| 서울특별시| 대전광역시| 경기도| 인천광역시| 경상남도| 강원도| 충청북도| 경상북도| 충청남도| 울산광역시| 대구광역시| 부산광역시'

In [45]:
count = 0
for i, one in tqdm(enumerate(adr)):
    multi_local = list(re.split(sep, one))
    if len(multi_local) == 2:
        count += 1
count

4364it [00:00, 337138.85it/s]


2

In [46]:
rep_dict = {" 광주광역시":"$광주광역시" ," 전라남도":"$전라남도"," 전라북도":"$전라북도"," 제주특별자치도":"$제주특별자치도"," 세종특별자치시":"$세종특별자치시"," 서울특별시":"$서울특별시"," 대전광역시":"$대전광역시"," 경기도":"$경기도"," 인천광역시":"$인천광역시"," 경상남도":"$경상남도"," 강원도":"$강원도"," 충청북도":"$충청북도"," 경상북도":"$경상북도"," 충청남도":"$충청남도"," 울산광역시":"$울산광역시"," 대구광역시":"$대구광역시"," 부산광역시":"$부산광역시"}

In [47]:
np.where(adr == '전라북도 순창군 풍산면 반월리 12-4 충청남도 공주시 탄천면 삼각리 523-2')

(array([2456], dtype=int64),)

In [48]:
adr[2456]

'전라북도 순창군 풍산면 반월리 12-4 충청남도 공주시 탄천면 삼각리 523-2'

In [49]:
count = 0
for i, one in tqdm(enumerate(adr)):
    for k, v in rep_dict.items():
        one = one.replace(k, v)
    multi_adr = list(one.split('$'))
    if len(multi_adr) == 2:
        count += 1
        for j, two in enumerate(multi_adr):
            if si[i] in two:
                adr[i] = multi_adr[j]
count

4364it [00:00, 159750.94it/s]


2

In [50]:
adr[2456]

'전라북도 순창군 풍산면 반월리 12-4'

### 도로명 주소 정리
- [길 숫자] 분리
- [로 숫자] 분리
- [로 숫자(번)길] 붙이기
- [앙 (숫자)길] 붙이기
- [장 (숫자)길] 붙이기
- [솜 (숫자)로] 붙이기
- [길 (숫자)번길] 붙이기
- [터 (숫자)길] 붙이기
- [개 (숫자)길] 붙이기

#### [분리] -길 숫자

In [51]:
np.where(adr == '전라북도 전주시 완산구 신촌3길24번지')

(array([15], dtype=int64),)

In [52]:
adr[15]

'전라북도 전주시 완산구 신촌3길24번지'

In [64]:
s = adr[15]
split1 = re.compile('([가-힣0-9]+길)([0-9]+)')
split1.sub('\g<1> \g<2>', s)

'전라북도 전주시 완산구 신촌3길 24번지'

In [65]:
count = 0
for i, one in enumerate(adr):
    if split1.search(one):
        adr[i] = split1.sub('\g<1> \g<2>', one)
        count += 1
count

71

In [66]:
adr[15]

'전라북도 전주시 완산구 신촌3길 24번지'

[분리] -로 숫자

In [53]:
np.where(adr == '경기도 이천시 마장면 중부고속도로82')

(array([85], dtype=int64),)

In [54]:
adr[85]

'경기도 이천시 마장면 중부고속도로82'

In [57]:
np.where(adr == '부산광역시 중구 중앙대로2 (중앙동7가)')

(array([1062], dtype=int64),)

In [58]:
adr[1062]

'부산광역시 중구 중앙대로2 (중앙동7가)'

In [69]:
s = adr[85]
split2 = re.compile('([가-힣0-9]+로)([0-9]+)')
split2.sub('\g<1> \g<2>', s)

'경기도 이천시 마장면 중부고속도로 82'

In [70]:
s = adr[1062]
split2.sub('\g<1> \g<2>', s)

'부산광역시 중구 중앙대로 2 (중앙동7가)'

In [71]:
count = 0
for i, one in enumerate(adr):
    if split2.search(one):
        adr[i] = split2.sub('\g<1> \g<2>', one)
        count += 1
count

366

In [72]:
adr[85]

'경기도 이천시 마장면 중부고속도로 82'

In [73]:
adr[1062]

'부산광역시 중구 중앙대로 2 (중앙동7가)'

#### [붙이기] -로 숫자(번)길

In [61]:
np.where(adr == '서울특별시 마포구 상암동 월드컵북로 58길 15 지하4층 18')

(array([1392], dtype=int64),)

In [62]:
adr[1392]

'서울특별시 마포구 상암동 월드컵북로 58길 15 지하4층 18'